Libreries 

In [ ]:
import src
from scipy.io import loadmat, whosmat
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt 
import os
import pandas as pd
import plotly.express as px
from src import config
from src import *
import re
from src import loadmatNina 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
import numpy as np
from itertools import combinations
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
import numpy as np
from itertools import combinations
from scipy.io import whosmat
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt




In [ ]:
# Choose the database to analyze
database = 'DB4'

data_path = f'data/{database}'

# Find the folder named with the convention s + "number"
folder = None
for item in os.listdir(data_path):
    if re.match(r'[sS]\d+', item) or re.match(r'Subject\d+', item):
        folder = item
        break

if folder:
    folder_path = os.path.join(data_path, folder)
    results = []

    # Iterate over all .mat files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.mat'):
            file_path = os.path.join(folder_path, file_name)
            info = whosmat(file_path)
            results.append((file_name, info))

    # Create a DataFrame to store the results
    data = {}
    for file_name, info in results:
        for item in info:
            if item[0] not in data:
                data[item[0]] = {}
            data[item[0]][file_name] = item[1:]

    df = pd.DataFrame(data)
    df = df.transpose()
    df.columns.name = 'File Name'

    print(df)
else:
    print("No folder found with the convention s + 'number'")

Functions

- For complete signal

In [ ]:
def calculate_emg_metrics(signal):
    """
    Calcula las métricas de una señal EMG.

    Parámetros:
    - signal: Array de NumPy que contiene la señal EMG.

    Retorna:
    - Un diccionario con las métricas calculadas.
    """
    # 1. Mean Absolute Value (MAV)
    mav = np.mean(np.abs(signal))
    
    # 2. Integrated Absolute Value (IAV)
    iav = np.sum(np.abs(signal))
    
    # 3. Temporal Difference (TD)
    td = np.sum(np.abs(np.diff(signal)))
    
    # 4. Root Mean Square (RMS)
    rms = np.sqrt(np.mean(signal**2))
    
    # 5. Mean Absolute Value Slope (MAVS)
    mavs = np.mean(np.abs(np.diff(signal)))
    
    # 6. Zero Crossings (ZC)
    zc = np.sum(np.diff(np.sign(signal)) != 0)
    
    # 7. Slope Sign Changes (SSC)
    diff_signal = np.diff(signal)
    ssc = np.sum((diff_signal[1:] * diff_signal[:-1]) < 0)
    
    # 8. Waveform Length (WL)
    wl = np.sum(np.abs(np.diff(signal)))
    
    # Retornar las métricas en un diccionario
    metrics = {
        "MAV": mav,
        "IAV": iav,
        "TD": td,
        "RMS": rms,
        "MAVS": mavs,
        "ZC": zc,
        "SSC": ssc,
        "WL": wl
    }
    
    return metrics


- For signal with means and standard deviation

In [ ]:
def calculate_emg_metrics_std(signal):
    """
    Calcula las métricas de una señal EMG.

    Parámetros:
    - signal: Array de NumPy que contiene la señal EMG.

    Retorna:
    - Un diccionario con las métricas calculadas.
    """
    if signal.ndim == 2:  # Si la señal tiene múltiples canales
        metrics_per_channel = [calculate_emg_metrics_std(signal[:, ch]) for ch in range(signal.shape[1])]
        averaged_metrics = {key: np.mean([m[key] for m in metrics_per_channel]) for key in metrics_per_channel[0]}
        return averaged_metrics
    
    # 1. Mean Absolute Value (MAV)
    mav = np.mean(np.abs(signal))
    mav_mean = np.mean(mav)
    mav_std = np.std(mav)

    # 2. Integrated Absolute Value (IAV)
    iav = np.sum(np.abs(signal))
    iav_mean = np.mean(iav)
    iav_std = np.std(iav)
    
    # 3. Temporal Difference (TD)
    td = np.sum(np.abs(np.diff(signal)))
    td_mean = np.mean(td)
    td_std = np.std(td)
    
    # 4. Root Mean Square (RMS)
    rms = np.sqrt(np.mean(signal**2))
    rms_mean = np.mean(rms)
    rms_std = np.std(rms)
    
    # 5. Mean Absolute Value Slope (MAVS)
    mavs = np.mean(np.abs(np.diff(signal)))
    mavs_mean = np.mean(mavs)
    mavs_std = np.std(mavs)
    
    # 6. Zero Crossings (ZC)
    zc = np.sum(np.diff(np.sign(signal)) != 0)
    zc_mean = np.mean(zc)
    zc_std = np.std(zc)
    
    # 7. Slope Sign Changes (SSC)
    diff_signal = np.diff(signal)
    ssc = np.sum((diff_signal[1:] * diff_signal[:-1]) < 0)
    ssc_mean = np.mean(ssc)
    ssc_std = np.std(ssc)
    
    # 8. Waveform Length (WL)
    wl = np.sum(np.abs(np.diff(signal)))
    wl_mean = np.mean(wl)
    wl_std = np.std(wl)
    
    # Retornar las métricas en un diccionario
    metrics = {
        "MAV": mav,
        "MAV_STD" : mav_std,
        "MAV_MEAN" : mav_mean,
        "IAV": iav,
        "IAV_STD" : iav_std,
        "IAV_MEAN" : iav_mean,
        "TD" : td,
        "TD_STD" : td_std,
        "TD_MEAN" : td_mean,
        "RMS": rms,
        "RMS_STD" : rms_std,
        "RMS_MEAN" : rms_mean,
        "MAVS": mavs,
        "MAVS_STD" : mavs_std,
        "MAVS_MEAN" : mavs_mean,
        "ZC": zc,
        "ZC_STD" : zc_std,
        "ZC_MEAN" : zc_mean,
        "SSC": ssc,
        "SSC_STD" : ssc_std,
        "SSC_MEAN" : ssc_mean,
        "WL": wl,
        "WL_STD" : wl_std,
        "WL_MEAN" : wl_mean
    }
    
    return metrics

- This functions calculate the metrics for channel and average the values for a complete result

In [ ]:
def calculate_emg_metrics_means(signal):
    """
    Calcula las métricas de una señal EMG. Si hay múltiples canales, calcula las métricas
    por canal y luego promedia los resultados.
    """
    if signal.ndim == 2:  # Si la señal tiene múltiples canales
        metrics_per_channel = [calculate_emg_metrics_means(signal[:, ch]) for ch in range(signal.shape[1])]
        averaged_metrics = {key: np.mean([m[key] for m in metrics_per_channel]) for key in metrics_per_channel[0]}
        return averaged_metrics
    
    # 1. Mean Absolute Value (MAV)
    mav = np.mean(np.abs(signal))
    
    # 2. Integrated Absolute Value (IAV)
    iav = np.sum(np.abs(signal))
    
    # 3. Temporal Difference (TD)
    td = np.sum(np.abs(np.diff(signal)))
    
    # 4. Root Mean Square (RMS)
    rms = np.sqrt(np.mean(signal**2))
    
    # 5. Mean Absolute Value Slope (MAVS)
    mavs = np.mean(np.abs(np.diff(signal)))
    
    # 6. Zero Crossings (ZC)
    zc = np.sum(np.diff(np.sign(signal)) != 0)
    
    # 7. Slope Sign Changes (SSC)
    diff_signal = np.diff(signal)
    ssc = np.sum((diff_signal[1:] * diff_signal[:-1]) < 0)
    
    # 8. Waveform Length (WL)
    wl = np.sum(np.abs(np.diff(signal)))
    
    # Retornar las métricas en un diccionario
    return {
        "MAV": mav,
        "IAV": iav,
        "TD": td,
        "RMS": rms,
        "MAVS": mavs,
        "ZC": zc,
        "SSC": ssc,
        "WL": wl
    }

Plots and metrics for complete grasp

In [ ]:
# Database name 
database = 'DB4'

# Full path to the database folder
data_path = os.path.abspath(os.path.join('data', database))

# List of subjects, generating names from 's1' to 's10'
subjects = [f's{i}' for i in range(1, 11)]

for subject in subjects:
    subject_dir = os.path.join(data_path, subject)
    
    # Archivos E1, E2, E3 para el sujeto actual
    for exercise in ["E1", "E2", "E3"]:
        filename = f"{subject.upper()}_{exercise}_A1.mat"
        file_path = os.path.join(subject_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"Archivo no encontrado: {file_path}")
            continue
        
        print(f"\nProcesando: {filename}")
        
        # Cargar datos del archivo .mat
        mat_data = src.loadmatNina(database, filename, subject=subject)
        
        # Verificar la estructura de mat_data
        print(f"Claves en mat_data: {mat_data.keys()}")
        
        # Obtener datos reetiquetados
        test_df, grasps_etiquetados = src.build_dataframe(
            mat_file=mat_data,
            database=database,
            filename=filename,
            rectify=False,
            normalize=True
        )
        
        # Iterar sobre grasps etiquetados
        for grasp in grasps_etiquetados:
            try:
                # Verificar si 'emg_data' está en mat_data
                if 'emg' not in mat_data:
                    raise KeyError(f"La clave 'emg_data' no está en mat_data. Claves disponibles: {mat_data.keys()}")
                
                # Obtener la señal EMG para el grasp reetiquetado
                emg_signal = mat_data['emg'][grasp]  # Ajusta esto según la estructura real
                
                # Calcular métricas de la señal EMG
                metrics = calculate_emg_metrics(emg_signal)
                
                # Imprimir las métricas
                print(f"\nMétricas para Grasp {grasp}:")
                for key, value in metrics.items():
                    print(f"{key}: {value:.4f}")
                
                # Graficar datos EMG para el grasp reetiquetado
                src.plot_emg_data(
                    database=database,
                    mat_file=mat_data,
                    grasp_number=grasp,
                    interactive=False,
                    include_rest=True,
                    use_stimulus=False,
                    addFourier=False,
                    padding=100,
                    title=f"{filename} - Grasp {grasp}"  # Usar el número del grasp actual
                )
            except KeyError as e:
                print(f"    Error: {str(e)}")
            except Exception as e:
                print(f"    Error procesando grasp {grasp}: {str(e)}")
                continue

Dataframe with metrics for a complete signal without discriminating by channel

In [ ]:
# Lista para almacenar todas las métricas
metrics_data = []

for subject in subjects:
    subject_dir = os.path.join(data_path, subject)
    
    for exercise in ["E1", "E2", "E3"]:
        filename = f"{subject.upper()}_{exercise}_A1.mat"
        file_path = os.path.join(subject_dir, filename)
        
        if not os.path.exists(file_path):
            continue  # Saltar archivos no encontrados
        
        mat_data = src.loadmatNina(database, filename, subject=subject)
        test_df, grasps_etiquetados = src.build_dataframe(
            mat_file=mat_data,
            database=database,
            filename=filename,
            rectify=False,
            normalize=True
        )
        
        for grasp in grasps_etiquetados:
            try:
                emg_signal = mat_data['emg'][grasp]
                metrics = calculate_emg_metrics(emg_signal)
                
                # Agregar metadatos + métricas a la lista
                metrics_data.append({
                    "subject": subject,
                    "exercise": exercise,
                    "filename": filename,
                    "grasp": grasp,
                    **metrics  # Desempaqueta todas las métricas
                })
                
            except Exception as e:
                print(f"Error en {filename} - Grasp {grasp}: {str(e)}")
                continue

# Crear DataFrame organizado
metrics_df = pd.DataFrame(metrics_data)

# Reordenar columnas (opcional)
column_order = ["subject", "exercise", "filename", "grasp"] + list(metrics.keys())
metrics_df = metrics_df[column_order]

print("\nDataFrame de Métricas:")
metrics_df

Dataframe with average of metrics for channels in each grasp

In [ ]:

# Lista para almacenar todas las métricas
metrics_data = []

for subject in subjects:
    subject_dir = os.path.join(data_path, subject)
    
    for exercise in ["E1", "E2", "E3"]:
        filename = f"{subject.upper()}_{exercise}_A1.mat"
        file_path = os.path.join(subject_dir, filename)
        
        if not os.path.exists(file_path):
            continue  # Saltar archivos no encontrados
        
        mat_data = src.loadmatNina(database, filename, subject=subject)
        test_df, grasps_etiquetados = src.build_dataframe(
            mat_file=mat_data,
            database=database,
            filename=filename,
            rectify=False,
            normalize=True
        )
        
        for grasp in grasps_etiquetados:
            try:
                emg_signal = mat_data['emg'][grasp]
                metrics = calculate_emg_metrics_means(emg_signal)
                
                # Agregar metadatos + métricas a la lista
                metrics_data.append({
                    "subject": subject,
                    "exercise": exercise,
                    "filename": filename,
                    "grasp": grasp,
                    **metrics  # Desempaqueta todas las métricas
                })
                
            except Exception as e:
                print(f"Error en {filename} - Grasp {grasp}: {str(e)}")
                continue

# Crear DataFrame organizado
metrics_df_2 = pd.DataFrame(metrics_data)

# Reordenar columnas (opcional)
column_order = ["subject", "exercise", "filename", "grasp"] + list(metrics_df_2.columns[4:])
metrics_df_2 = metrics_df_2[column_order]

print("\nDataFrame de Métricas:")
print(metrics_df_2)



Dataframe with mean and standard deviation

In [ ]:
# Lista para almacenar todas las métricas
metrics_data = []

for subject in subjects:
    subject_dir = os.path.join(data_path, subject)
    
    for exercise in ["E1", "E2", "E3"]:
        filename = f"{subject.upper()}_{exercise}_A1.mat"
        file_path = os.path.join(subject_dir, filename)
        
        if not os.path.exists(file_path):
            continue  # Saltar archivos no encontrados
        
        mat_data = src.loadmatNina(database, filename, subject=subject)
        test_df, grasps_etiquetados = src.build_dataframe(
            mat_file=mat_data,
            database=database,
            filename=filename,
            rectify=False,
            normalize=True
        )
        
        for grasp in grasps_etiquetados:
            try:
                emg_signal = mat_data['emg'][grasp]
                metrics = calculate_emg_metrics_std(emg_signal)
                
                # Agregar metadatos + métricas a la lista
                metrics_data.append({
                    "subject": subject,
                    "exercise": exercise,
                    "filename": filename,
                    "grasp": grasp,
                    **metrics  # Desempaqueta todas las métricas
                })
                
            except Exception as e:
                print(f"Error en {filename} - Grasp {grasp}: {str(e)}")
                continue

# Crear DataFrame organizado
metrics_df_std = pd.DataFrame(metrics_data)

# Reordenar columnas (opcional)
column_order = ["subject", "exercise", "filename", "grasp"] + list(metrics.keys())
metrics_df_std = metrics_df_std[column_order]

print("\nDataFrame de Métricas:")
metrics_df_std

Dataframe for every channels of data base

In [ ]:
# Lista para almacenar todos los DataFrames
all_dataframes = []

# Encontrar todas las carpetas que coincidan con el patrón s + "number" o Subject + "number"
for folder in os.listdir(data_path):
    if re.match(r'[sS]\d+', folder) or re.match(r'Subject\d+', folder):
        folder_path = os.path.join(data_path, folder)
        
        # Iterar sobre todos los archivos .mat en la carpeta
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.mat'):
                file_path = os.path.join(folder_path, file_name)
                
                # Cargar el archivo .mat
                try:
                    mat_data = src.loadmatNina(database, file_name, subject=folder)
                except Exception as e:
                    print(f"Error al cargar {file_name}: {str(e)}")
                    continue
                
                # Procesar el archivo con src.build_dataframe
                try:
                    test_df, grasps = src.build_dataframe(
                        mat_file=mat_data,
                        database=database,
                        filename=file_name,
                        rectify=False,
                        normalize=True
                    )
                    
                    # Agregar una columna con el sujeto (folder) al DataFrame
                    test_df['subject'] = file_name
                    
                    # Agregar el DataFrame a la lista
                    all_dataframes.append(test_df)

                except Exception as e:
                    print(f"Error al procesar {file_name}: {str(e)}")
                    continue

# Concatenar todos los DataFrames en uno solo
if all_dataframes:  
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Mostrar el DataFrame combinado
    print("\nDataFrame Combinado:")
    display(combined_df)  

else:
    print("Advertencia: No se generaron DataFrames. Verifica los datos de entrada.")

Dataframe with metrics for channel

In [ ]:
# Lista para almacenar las métricas por canal
metrics_data = []

# Iterar sobre cada sujeto y relabeled (o stimulus)
for (subject, relabeled), group in combined_df.groupby(['subject', 'relabeled']):  # Cambia 'relabeled' por 'stimulus' si es necesario
    # Iterar sobre cada canal
    for channel in group.columns:  # Itera sobre todas las columnas
        if channel.startswith('Channel'):  # Filtra solo las columnas de canales
            # Obtener la señal del canal
            channel_signal = group[channel].values
            
            # Calcular métricas para el canal actual
            metrics = calculate_emg_metrics(channel_signal)
            
            # Agregar metadatos + métricas a la lista
            metrics_data.append({
                "subject": subject,
                "relabeled": relabeled,  # Cambia 'relabeled' por 'stimulus' si es necesario
                "channel": channel,
                **metrics  # Desempaqueta todas las métricas
            })

# Crear DataFrame con las métricas
metrics_df = pd.DataFrame(metrics_data)

# Reordenar columnas (opcional)
column_order = ["subject", "relabeled", "channel"] + list(metrics.keys())
metrics_df = metrics_df[column_order]

# Mostrar el DataFrame de métricas
print("\nDataFrame de Métricas por Canal, Sujeto y Relabeled:")
display(metrics_df)  # Usar display en Jupyter Notebook


In [ ]:
grouped_df = metrics_df.drop(columns = ['channel'])

df_mean = grouped_df.groupby(['subject','relabeled']).mean()
df_std = grouped_df.groupby(['subject','relabeled']).std()

df_mean.columns = [f"{col} mean" for col in df_mean.columns]
df_std.columns = [f"{col} std" for col in df_std.columns]

df_result = df_mean.merge(df_std, on=['subject', 'relabeled']).reset_index()

# Agrupar por 'subject' y 'relabeled' y calcular el promedio y la desviación estándar de las métricas
# grouped_df.columns = ['subject', 'relabeled'] + [f"{col} mean" for col in grouped_df.columns[2:]]

display(df_result)

Dendogram for grasp 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler

# Seleccionar solo las columnas de características
features = df_result.iloc[:, 2:]  # Omitimos 'subject' y 'relabeled'

# Normalizar los datos para mejor visualización
df_scaled = StandardScaler().fit_transform(features)

# Aplicar clustering jerárquico
linked = sch.linkage(df_scaled, method='ward')

# Crear el dendrograma
plt.figure(figsize=(20, 10))
sch.dendrogram(linked, labels=df_result['relabeled'].values, leaf_rotation=90, leaf_font_size=8)
plt.title("Dendrograma basado en la relabeled")
plt.xlabel("Clusters")
plt.ylabel("Distancia Euclidiana")
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

# Cargar los datos (asumiendo que ya tienes el DataFrame 'data')
# Agrupar por 'relabeled' y calcular media y desviación estándar
grouped = df_result.select_dtypes(include=['number']).groupby(df_result['relabeled']).agg(['mean', 'std'])
display(grouped)
# Aplanar los nombres de columnas (opcional)
grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]

# Normalizar los datos para evitar que las magnitudes afecten la distancia
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(grouped)

# Aplicar clustering jerárquico
linked = sch.linkage(scaled_features, method='ward')

# Crear el dendrograma
plt.figure(figsize=(12, 6))
sch.dendrogram(linked, labels=grouped.index.tolist(), leaf_rotation=90, leaf_font_size=8)
plt.title("Dendrograma basado en media y desviación estándar por tipo de grasp")
plt.xlabel("Grasps")
plt.ylabel("Distancia Euclidiana") 
plt.show()


In [ ]:
# 1. Calcular el promedio de las métricas por canal
metrics_columns = [col for col in metrics_df.columns if col not in ["subject", "relabeled", "channel"]]
average_metrics_df = metrics_df.groupby('channel')[metrics_columns].mean().reset_index()
display(average_metrics_df)

# 2. Preparar los datos para el clustering
X = average_metrics_df[metrics_columns].values

# 3. Calcular la matriz de distancias y realizar el clustering jerárquico
Z = linkage(X, method='ward')  # 'ward' es un método común para clustering jerárquico

# 4. Graficar el dendrograma con ajustes para mejor visualización
plt.figure(figsize=(15, 8)) 
plt.title('Dendrograma de Canales EMG (Promedio de Métricas)', fontsize=16, pad=20)
plt.xlabel('Canales', fontsize=14)
plt.ylabel('Distancia', fontsize=14)

# Ajustar el dendrograma para que las etiquetas no se superpongan
dendrogram(
    Z,
    labels=average_metrics_df['channel'].values,  # Etiquetas de los canales
    leaf_rotation=90, 
    leaf_font_size=12,  
    color_threshold=0.7 * max(Z[:, 2]),  # Umbral para colorear los clusters
)

plt.tight_layout()  
plt.show()

In [ ]:
# 1. Calcular el promedio y la desviación estándar de las métricas por canal
metrics_columns = [col for col in metrics_df.columns if col not in ["subject", "relabeled", "channel"]]

agg_metrics_df = metrics_df.groupby('channel')[metrics_columns].agg(['mean', 'std']).reset_index()

# Aplanar las columnas para facilitar el acceso
agg_metrics_df.columns = ['_'.join(col).strip('_') for col in agg_metrics_df.columns]

display(agg_metrics_df)

# 2. Preparar los datos para el clustering usando solo los promedios
X = agg_metrics_df[[col for col in agg_metrics_df.columns if col.endswith('_mean')]].values

# 3. Calcular la matriz de distancias y realizar el clustering jerárquico
Z = linkage(X, method='ward')  # 'ward' es un método común para clustering jerárquico

# 4. Graficar el dendrograma con ajustes para mejor visualización
plt.figure(figsize=(15, 8)) 
plt.title('Dendrograma de Canales EMG (Promedio de Métricas)', fontsize=16, pad=20)
plt.xlabel('Canales', fontsize=14)
plt.ylabel('Distancia', fontsize=14)

# Ajustar el dendrograma para que las etiquetas no se superpongan
dendrogram(
    Z,
    labels=agg_metrics_df['channel'].values,  # Etiquetas de los canales
    leaf_rotation=90, 
    leaf_font_size=12,  
    color_threshold=0.7 * max(Z[:, 2]),  # Umbral para colorear los clusters
)

plt.tight_layout()  
plt.show()

In [ ]:
# Usamos solo las métricas para el clustering, excluyendo las columnas de metadatos
metrics_columns = [col for col in metrics_df.columns if col not in ["subject", "relabeled", "channel"]]
X = metrics_df[metrics_columns].values

# Calcular la matriz de distancias y realizar el clustering jerárquico
Z = linkage(X, method='ward')  # 'ward' es un método común para clustering jerárquico

# Graficar el dendrograma con ajustes para mejor visualización
plt.figure(figsize=(20, 10))  # Aumentar el tamaño de la figura
plt.title('Dendrograma de Canales EMG', fontsize=16, pad=20)
plt.xlabel('Canales', fontsize=14)
plt.ylabel('Distancia', fontsize=14)

# Ajustar el dendrograma para que las etiquetas no se superpongan
dendrogram(
    Z,
    labels=metrics_df['channel'].values,  # Etiquetas de los canales
    leaf_rotation=90,  # Rotar las etiquetas 90 grados
    leaf_font_size=10,  # Tamaño de la fuente de las etiquetas
    color_threshold=0.7 * max(Z[:, 2]),  # Umbral para colorear los clusters
)

plt.tight_layout()  # Ajustar el layout para evitar cortes en las etiquetas
plt.show()